In [47]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time
import numpy as np
from my_keys import *

In [46]:
# get timestamp in correct format for tomorrow 0,12 and 24 
def get_tomorrow():
    #make tomorow day, by adding one day to today
    date_tomorrow = date.today() + timedelta(days=1)
    #make three times: 00:00:00,12:00:00,23:59:59
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)
    #combine tomorow day and the times
    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    #return three tomorow times
    return datetime_start,datetime_middle,datetime_end

In [48]:
# weather_forcast function that het a list of citie names and return the weather_forcast for tomorow
def weather_forcast(cities_df,openweather_api_key):
   
    
    cities_list  = []#i want to use the cities information from this api, but it is not a good source i think
    weather_list = []#a list to stor all citys weathers
    error =[]# i try to catch errors and manage them, its just to control them, ill make it right later
    dt_start,dt_middle,dt_end = get_tomorrow() #generate the datetime format variabels fro tommorrow times
    #a loop for all citiess in city dataframe
    for index, city in cities_df.iterrows():
        
        #sent a requiest to openweather with city name and mykey
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city['city']}&appid={openweather_api_key}&units=metric")
        
        #check if the responce is not ok
        if city_r.status_code != 200:
            error.append(city+" got errpr code: "+ str(city_r.status_code)) #add the  error code with the message include the name of city
            continue #when this city has error just continue the for loop
        
        city_j = city_r.json() #convert the reponse to json
        
        cities_list.append(city_j['city']) #add citys information to cities_list
        
        # the api gives us a list of weather detail for every 3 hours for next 5 days and store each details in a dict 
        for detail in city_j['list']:
            weather_details_dict = {} #create an empty weather dict
            
            #choose that details that we want in the key names
            weather_details_dict["temperature"] = detail['main']['temp']
            weather_details_dict["temp_min"] = detail['main']['temp_min']
            weather_details_dict["temp_max"] = detail['main']['temp_max']
            weather_details_dict["feels_like"] = detail['main']['feels_like']
            weather_details_dict["pressure"] = detail['main']['pressure']
            weather_details_dict["humidity"] = detail['main']['humidity']
            weather_details_dict["weather"] = detail['weather'][0]['main']
            weather_details_dict["weather_description"] = detail['weather'][0]['description']
            weather_details_dict["clouds"] = detail['clouds']['all']
            weather_details_dict["wind_speed"] = detail['wind']['speed']
            weather_details_dict["date_time_3h"] = detail['dt_txt']
            
            weather_details_dict["city_id"] = city['city_id'] # use the city_id from cities_df to use as forigen key
            weather_list.append(weather_details_dict) # add dict to weather_list
    
    
    df_weather = pd.DataFrame(weather_list) #convert the weather_list to pandas dataframe
    df_weather['date_time_3h'] = pd.to_datetime(df_weather['date_time_3h']) #convert the date_time_3h to pdatetime format
    df_weather = df_weather.loc[lambda df_ : (df_['date_time_3h'] >= dt_start) &  (df_['date_time_3h'] <= dt_end)] #i select just the tomorros datas
    
    
    df_cities = pd.json_normalize(cities_list) #convert df_cities to pandas data frame
    df_cities.rename(columns={'id':'city_id'}, inplace=True) # rename the id column to city_id
    
    #retuen both dafata frames and error list
    return df_weather,df_cities,error

In [49]:
import re
from bs4 import BeautifulSoup

# function that collect the data for each city in cities_list from the wikipedia and wft-geo-db.p.rapidapi.com api
def get_cities(cities_list,RapidAPI_key):
    #make a empty data frame with the columns that we want
    city_columns= ['id', 'wikiDataId', 'type', 'name', 'country', 'countryCode', 'region', 'mayor', 'elevationMeters', 'latitude', 'longitude',	'population', 'timezone']
    cities_df = pd.DataFrame(columns= city_columns)
    error =[]# i try to catch errors and manage them, its just to control them, ill make it right later
    #loop on all cities in cities_list
    for city in cities_list :
        #collect the wiki id for cities an the name of mayoe for ich city
        wiki_url = f"https://en.wikipedia.org/wiki/{city}" #make wikipedia url with the city name
        wiki_city_r = requests.get(wiki_url) # send a request to wikipedia 
        
        if wiki_city_r.status_code != 200:
            error.append(city+" got errpr code: "+ str(wiki_city_r.status_code)+"from wikipedia") #add the  error code with the message include the name of city
            continue #when this city has error just continue the for loop
        
        wiki_city_soup = BeautifulSoup(wiki_city_r.content, "html.parser") #convert the response to BeautifulSoup variable
        #find the html a element that its string contains 'Mayor'  then find its grandparent then find the name of mayor in its place
        try: city_mayor = wiki_city_soup.find('a' , text = re.compile(".*Mayor.*")).parent.parent.find('td').find('a').string 
        except: city_mayor = 'Unknown' # managing error, when ther is no mayor name
        # two selects to find the wikiId for each city
        #  city_wiki_id = wiki_city_soup.select('li#t-wikibase.mw-list-item a')[0]['href'].split('/')[-1]
        city_wiki_id = wiki_city_soup.select('div#mw-navigation div#mw-panel nav#p-tb div.vector-menu-content ul.vector-menu-content-list li#t-wikibase.mw-list-item a')[0]['href'].split('/')[-1]
        
        #generate a url to get the city information from /wft-geo-db.p.rapidapi.com
        url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{city_wiki_id}"
        headers = {
	                "X-RapidAPI-Key": RapidAPI_key, #######################
	                "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
                    }
        city_informatin_r = requests.request("GET", url, headers=headers)
        
        if wiki_city_r.status_code != 200:
            error.append(city+" got errpr code: "+ str(wiki_city_r.status_code)+"from wft-geo-db.p.rapidapi.com") #add the  error code with the message include the name of city
            continue #when this city has error just continue the for loop
        
        city_informatin_j = city_informatin_r.json() #covert responce to json
        city_informatin_df = pd.json_normalize(city_informatin_j['data']) #convert json to pandas dataframe
        city_informatin_df["mayor"] = city_mayor #add the mayor name in city_informatin_df
        city_informatin_df = city_informatin_df[city_columns] #select the columns that we want
        cities_df = pd.concat([cities_df,city_informatin_df]) #concat the city information with the cities_df
    #make a rename dict to rename the columns as we want    
    city_columns_rename= {'id' : 'city_id', 'wikiDataId' : 'wiki_data_id', 'name' : 'city', 'elevationMeters' : 'elevation', 'countryCode' : 'country_code'}
    cities_df.rename(columns = city_columns_rename, inplace=True)#aplly rename with our rename dict
    
    cities_df['population'] = pd.to_numeric(cities_df['population']) # convert 'population' column to numeric
    cities_df['city_id'] = pd.to_numeric(cities_df['city_id']) # convert 'city_id' column to numeric
    cities_df.reset_index(drop=True, inplace= True) #rest index for cities_df
    return cities_df,error

In [50]:
#take the key as intertaudi 
#function get all aiports for ich city in cities_df
def get_citys_airports(cities_df,RapidAPI_key):
    #make a empty data frame with the columns that we want
	columns=['icao','iata','name','short_name','longitude_deg','iso_country','latitude_deg','city_name','city_id']
	airports_df = pd.DataFrame(columns=columns)
	error =[]# i try to catch errors and manage them, its just to control them, ill make it right later
	# loop for every cities in cities_df
	for index,city in cities_df.iterrows():
		#generate a request for aerodatabox.p.rapidapi.com, that give us all airport for that city name
		url = "https://aerodatabox.p.rapidapi.com/airports/search/term"	
		querystring = {"q":city['city'],"limit":"10"}
		headers = {
			"X-RapidAPI-Key": RapidAPI_key,
			"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
			}
		airports_r = requests.request("GET", url, headers=headers, params=querystring)

		if airports_r.status_code != 200:
			error.append(city['city'] + " got errpr code: "+ str(airports_r.status_code)) #add the  error code with the message include the name of city
			continue #when this city has error just continue the for loop
		
		airports_j = airports_r.json()#convert to json
		if len(airports_j['items'])==0: #check if the city has no airport
			error.append(city['city'] + " has no airport") #add the  error  with the message include the name of city
			continue #when this city has error just continue the for loop

		df_airports =pd.json_normalize(airports_j['items'])#convert to pandas dataframe
		df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == city['country_code']] #ignore the same name cities of other country
		rename_dict = {'location.lat' : 'latitude_deg', 'location.lon' : 'longitude_deg', 'municipalityName': 'city_name', 'countryCode' : 'iso_country',
								'shortName' : 'short_name'}#make a rename dict that wee want to rename
		df_airports.rename(columns=rename_dict, inplace=True) #aplly the rename
		df_airports['city_id'] = pd.to_numeric(city['city_id'])# convert 'city_id' column to numeric
		df_airports = df_airports[columns]#select the columns that we want
		airports_df = pd.concat([airports_df,df_airports])#concat the airports for each city
	airports_df.reset_index(drop=True, inplace=True)
	return airports_df,error

In [51]:
 #function get all arrival flights for every airport in airports_df 
def get_arrival_flights(airports,RapidAPI_key):
    #make a empty data frame with the columns that we want
    columns=['flight_NO', 'flight_status', 'dep_airport', 'dep_airport_icao', 'dep_airport_iata','sched_arr_loc_time',
                                    'sched_arr_Utc', 'terminal' , 'airline','aircraf', 'icao']
    arrival_flights_df = pd.DataFrame(columns=columns)
    #make a rename dict that wee want to rename
    rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                        'movement.airport.name' : 'dep_airport','movement.airport.icao' : 'dep_airport_icao', 'movement.airport.iata':'dep_airport_iata', 
                        'movement.scheduledTimeLocal' : 'sched_arr_loc_time', 'movement.scheduledTimeUtc' : 'sched_arr_Utc', 'movement.terminal': 'terminal',
                        'airline.name':'airline', 'aircraft.model' :'aircraf'}
    error = []# i try to catch errors and manage them, its just to control them, ill make it right later
    dt_start,dt_middle,dt_end = get_tomorrow() #generate the datetime format variabels fro tommorrow times
    #loop for all airports in airports
    for index,airport in airports.iterrows():
        #generate a request for aerodatabox.p.rapidapi.com, that give us all arrival flights for that airport icao
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport['icao']}/{dt_middle}/{dt_end}"
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}
        headers = {
            "X-RapidAPI-Key": RapidAPI_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        arrivals_r = requests.request("GET", url, headers=headers, params=querystring)
        
        if arrivals_r.status_code != 200:
            error.append(airport['icao'] + " got errpr code: " + str(arrivals_r.status_code))#add the error code with the message include the icao
            continue #when this airport has error just continue the for loop
        
        arrivals_j = arrivals_r.json() #convert to json
        
        if len(arrivals_j['arrivals'])==0: #check if the airport has no flight
            error.append(airport['icao'] + " has no flight")#add the error with the message include the icao
            continue #when this airport has error just continue the for loop
        
        #convert json to pandas dataframe andd sort it by movement.scheduledTimeLocal
        arrivals_df = pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

        
        # clean df
        
        #some air ports dont have terminal, i try to catch the crash
        try: arrivals_df = arrivals_df[['number', 'status', 
                        'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                        'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc', 'movement.terminal'
                        'airline.name', 'aircraft.model']]
        except: arrivals_df = arrivals_df[['number', 'status', 
                        'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                        'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc',
                        'airline.name', 'aircraft.model']]
        
        arrivals_df.rename(columns=rename_dict , inplace=True)#apply rename 
        arrivals_df['icao'] = airport['icao']#  add the icaro column with the airpot icaro to use as forigen key
    
        arrival_flights_df = pd.concat([arrival_flights_df,arrivals_df])#concat eacht airport flights to resault dafataframe
    arrival_flights_df.reset_index(drop=True, inplace=True)
    return arrival_flights_df,error


In [52]:
#function get extra information for ich city in cities_df
def get_citys_informations(cities_df, RapidAPI_key):
    #make three empty data frame with the columns that we want for hotels, landmark and transport
	columns=['id','name','type','latitude','longitude','city_id']
	rename_dict = {'geoId' : 'id'}#make a rename dict that wee want to rename
	hotels = pd.DataFrame(columns=columns)
	hotels_df = pd.DataFrame(columns=columns)
	landmarks= pd.DataFrame(columns=columns)
	landmark_df = pd.DataFrame(columns=columns)
	transports = pd.DataFrame(columns=columns)
	transport_df = pd.DataFrame(columns=columns)
	error =[]# i try to catch errors and manage them, its just to control them, ill make it right later
	# loop for every cities in cities_df
	for index,city in cities_df.iterrows():
		#generate a request for "hotels4.p.rapidapi.com", that give us all airport for that city name
		
		url = "https://hotels4.p.rapidapi.com/locations/v2/search"
		querystring = {"query":city['city'],"locale":"en_US","currency":"USD"}

		headers = {
				"X-RapidAPI-Key": RapidAPI_key,
				"X-RapidAPI-Host": "hotels4.p.rapidapi.com"
				}
		city_extra_information_r = requests.request("GET", url, headers=headers, params=querystring)

		if city_extra_information_r.status_code != 200:
			error.append(city['city'] + " got errpr code: "+ str(city_extra_information_r.status_code)) #add the  error code with the message include the name of city
			continue #when this city has error just continue the for loop
		
		city_extra_information_j = city_extra_information_r.json()#convert to json
		city_hotels = city_extra_information_j['suggestions'][1]['entities']
		city_landmark = city_extra_information_j['suggestions'][2]['entities']
		city_transport = city_extra_information_j['suggestions'][3]['entities']
		if len(city_hotels)==0: #check if the city has no hotel
			error.append(city['city'] + " has no hotels") #add the  error  with the message include the name of city
		else :
			hotels = pd.json_normalize(city_hotels)
			hotels['city_id'] = pd.to_numeric(city['city_id'])# convert 'city_id' column to numeric 
			hotels.rename(columns=rename_dict, inplace=True) #aplly the rename
			hotels = hotels[columns]#select the columns that we want	
  
		if len(city_landmark)==0: #check if the city has no hotel
			error.append(city['city'] + " has no landmark") #add the  error  with the message include the name of city
		else :
			landmarks = pd.json_normalize(city_landmark)
			landmarks['city_id'] = pd.to_numeric(city['city_id'])# convert 'city_id' column to numeric
			landmarks.rename(columns=rename_dict, inplace=True) #aplly the rename
			landmarks = landmarks[columns]#select the columns that we want		

		if len(city_transport)==0: #check if the city has no hotel
			error.append(city['city'] + " has no transport") #add the  error  with the message include the name of city
		else :
			transports = pd.json_normalize(city_transport)
			transports['city_id'] = pd.to_numeric(city['city_id'])# convert 'city_id' column to numeric 
			transports.rename(columns=rename_dict, inplace=True) #aplly the rename
			transports = transports[columns]#select the columns that we want
   
		rename_dict = {'geoId' : 'id'}#make a rename dict that wee want to rename
		# df_airports.rename(columns=rename_dict, inplace=True) #aplly the rename

		hotels_df = pd.concat([hotels_df,hotels])#concat the hotels for each city
		landmark_df = pd.concat([landmark_df,landmarks])#concat the hotels for each city
		transport_df = pd.concat([transport_df,transports])#concat the hotels for each city
	hotels_df.reset_index(drop=True, inplace=True)	
	landmark_df.reset_index(drop=True, inplace=True)
	transport_df.reset_index(drop=True, inplace=True)
	return hotels_df,landmark_df,transport_df,error

In [57]:
#this function will manage the update of all tables and datas, it takes a list of cities and a connection, and then 
# try to add new cities to the cities_df and new city information in airports, hotels, transports ans landmarks and then 
# it will push the new data to sql by connection that took.
def update_information(cities_list, con):
    #read cities table from the data base
    excist_cities_df = pd.read_sql("select * from cities", con)
    #find new cities that arent in database
    new_cities_list = [city for city in cities_list if (city not in excist_cities_df['city'].tolist())]
    #generate new information for new cities
    cities_df, cities_error = get_cities(new_cities_list,my_RapidAPI_key)
    hotels_df,landmark_df,transport_df,error = get_citys_informations(cities_df, my_RapidAPI_key)
    
    #and push the data in to cieties information tables
    cities_df['elevation'] =cities_df['elevation'].fillna(value=0)#fill na s with 0
    cities_df.to_sql('cities', con=con, if_exists='append', index=False)
    hotels_df.to_sql('hotels', con=con, if_exists='append', index=False)
    landmark_df.to_sql('landmarks', con=con, if_exists='append', index=False)
    transport_df.to_sql('transports', con=con, if_exists='append', index=False)
    
    #generate new information for new airports and push them to arrivals table
    airports_df, airports_error = get_citys_airports(cities_df,my_RapidAPI_key)
    airports_df.to_sql('airports', con=con, if_exists='append', index=False)
    
    #read all cities and airports from sql tables
    excist_cities_df = pd.read_sql("select * from cities", con)
    excist_airports_df = pd.read_sql("select * from airports", con)
    
    #generate new weather information for all cities and push them in sql database
    weather_data, cities_df_2 ,weather_error= weather_forcast(excist_cities_df,my_openweather_api_key)
    weather_data.to_sql('weather', con=con, if_exists='append', index=False)
    
    #generate new arrival flights information for all arports and push them in sql database
    arrival_flights_df,flights_error = get_arrival_flights(excist_airports_df,my_RapidAPI_key)
    arrival_flights_df.fillna(value ="Unknown").to_sql('arrivals', con=con, if_exists='append', index=False)
    
    

In [55]:
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
update_information(cities,con)

the next codes are template and test and try codes

In [21]:
#PART1
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']
cities_df, cities_error = get_cities(cities)

In [530]:
#PART2
weather_data, cities_df_2 ,weather_error= weather_forcast(cities_df)

In [11]:
#PART3
airports_df, airports_error = get_citys_airports(cities_df)

In [589]:
# airports_df

In [ ]:
#PART4
arrival_flights_df,flights_error = get_arrival_flights(airports_df)

In [35]:
#PART5
hotels_df,landmark_df,transport_df,error = get_citys_informations(cities_df, my_RapidAPI_key)

In [ ]:
transport_df

save the data in mysql

In [13]:
import pandas as pd
import sqlalchemy
import pymysql

In [14]:
#all this variables filled in my_keys.py
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [18]:
cities_df['elevation'] =cities_df['elevation'].fillna(value=0)#fill na s with 0
excist_cities_df = pd.read_sql("select * from cities", con)
new_cities_df = cities_df.loc[~cities_df['city'].isin(excist_cities_df['city'])]
new_cities_df.to_sql('cities', con=con, if_exists='append', index=False)

0

In [587]:
airports_df.to_sql('airports', con=con, if_exists='append', index=False)

16

In [586]:
weather_data.to_sql('weather', con=con, if_exists='append', index=False)

72

In [588]:
arrival_flights_df.fillna(value ="Unknown").to_sql('arrivals', con=con, if_exists='append', index=False)

3568

In [ ]:
pd.read_sql("select * from cities", con)